# dask-awkward IO Tutorial

_last updated 2024-01-08_

In [1]:
from __future__ import annotations
import awkward
import dask_awkward
print("package versions:")
print(f"awkward:       {awkward.__version__}")
print(f"dask-awkward:  {dask_awkward.__version__}")

package versions:

awkward:       2.5.1

dask-awkward:  2024.1.0

## I. The Basics

### Data on disk

`dask-awkward` supports a number of file formats out-of-the-box. Those include:

- Parquet
- JSON
- Plain text


> _Note_: The [uproot project](https://github.com/scikit-hep/uproot5) provides the `uproot.dask` module for reading the ROOT file format into dask-awkward collections. This tutorial will focus on file formats that have support baked into `dask-awkward`.

Since this is Dask, data on disk is _staged_ for reading when using a dask-awkward read function. Data will not be read until a `compute` or (`persist`) call is reached.

**Note**: we can also create dask-awkward `Array` collections from other Dask collections (bag, array, dataframe, delayed) or from data that already exists in memory (awkward arrays and Python lists)

Let's jump into a quick example reading a Parquet dataset.

In [2]:
import dask_awkward as dak
pq_dir = os.path.join("data", "parquet")
dataset = dak.from_parquet(pq_dir)

In [3]:
dataset

dask.awkward<from-parquet, npartitions=4>

By default the `dak.from_parquet` function will partition by file. Since the directory has four files, we will have four partitions in our Dask collection.

Without computing anything, the `from_parquet` function has extracted some metadata, so we are able to get a peek at the structure of what the eventual awkward array would be if we did compute this collection:

In [4]:
dataset._meta

<Array-typetracer [...] type='## * {type: string, scoring: var * {player: s...'>

You'll notice this dataset has two top level fields:
- `type`
- `scoring`

Inside of the `scoring` field (or Parquet column) are three subfields (`scoring` is a `Record` in awkward array terminology):

- `player`
- `basket`
- `distance`

We can also see that for each element in the top level array, we have exactly one entry for the `type` field, and some variable (showing array raggedness) number of `scoring` entries.

The data we have here is some made up data about basketball games/matches. Each game is labeled as either a "friendly" match or a "league" match. Each game has some number of total scores, each score being made by some player as some type of basket at some distance. The raggedness of the array comes from each match having a different total number of scores.

Since this first section of the tutorial is meant to show the basics of the IO functions, we won't worry too much about the details of the dataset, but we will revisit the structure in the next section!

Since this tutorial is using a small toy dataset we can easily compute it quickly to see a concrete awkward array:

In [5]:
computed_dataset = dataset.compute()

In [6]:
computed_dataset

<Array [{type: 'league', ...}, ..., {...}] type='200 * {type: string, scori...'>

With parquet, we can restrict our data reading to only grab a specific set of columns from the files. In this toy dataset we're working with, if we only care about the specific players which did some scoring, we can specific that:

In [7]:
dataset = dak.from_parquet(pq_dir, columns=["scoring.player"])

In [8]:
dataset._meta

<Array-typetracer [...] type='## * {scoring: var * {player: string}}'>

Notice that when we peek at the metadata now, we see our array is going to contain less information, as expected! If we tied to access one of the fields we didn't request, we'd hit an `AttributeError` (before compute time!). Since we are able to track metadata at graph construction time, we can fail as early as possible

In [9]:
dataset.scoring.distance

AttributeError: distance not in fields.

Let's go back to the original dataset and save it to JSON after repartitioning the collection:

In [10]:
dataset = dak.from_parquet(pq_dir)
smaller_partition_dataset = dataset.repartition(15)
dak.to_json(smaller_partition_dataset, os.path.join("data", "json"))

`dask-awkward`'s `to_*` functions have a bit of special treatmeant compared to other dask-awkward functions. They are the only parts of dask-awkward that are _eagerly_ computed. The `to_*` functions have a `compute=` argument that defaults to `True`. If you'd like to stage a data writing step without compute, you can write:

In [11]:
write_it = dak.to_json(smaller_partition_dataset, os.path.join("data", "json2"), compute=False)

In [12]:
write_it

dask.awkward<to-json, type=Scalar, dtype=float64>

Notice that the `write_it` object is a dask-awkward `Scalar` collection that can be computed.

In [13]:
write_it.compute()

Now we can reload our data with `dak.from_json`. Realistically, taking data stored in parquet to then save it as JSON to be read later is likely a bad idea! But we're just doing this to show example usage of the dask-awkward API.

In [14]:
dataset = dak.from_json(os.path.join("data", "json"))

In [15]:
dataset

dask.awkward<from-json-files, npartitions=15>

## II. Column (buffer) optimization

Dask workflows can be separated into two stages: first is task graph construction, and second is task graph execution. During task graph construction we are able to track metadata about our awkward array collections; with that metadata knowledge we are able, just before execution time, to know which parts of the Array are necessary to complete a computation. This is possible by running the task graph on a metadata only version of the arrays. When we run the metadata task graph, components of the data-less array are "touched" by the execution of the graph, and when that happens we know that's a part of the data on disk that needs to be read. 

Let's look at a quick example with Parquet. Recall the dataset from the previou section. We have these columns:

- `type`
- `scoring.player`
- `scoring.basket`
- `scoring.distance`

If we want to calculate the average distance of each scored basket during each game, ignoreing all freethrows, we can calculate that like so:

In [16]:
dataset = dak.from_parquet(pq_dir)
free_throws = dak.str.match_substring(dataset.scoring.basket, "freethrow")
distances = dataset.scoring.distance[free_throws == False]
result = dak.mean(distances, axis=1)

The `result` will be the average distance of each non-free-throw shot. Notice we only used two of the four columns: `scoring.basket` and `scoring.distance`, If we wanted to be explicit about it, we could use the `columns=` argument in the `dak.from_parquet` call. But we can also just rely on dask-awkward to do this for us! The columns/buffer optimization will detect that the graph is only going to need those columns, rewriting the internal `ak.from_parquet` call at the node in the task graph that actually reads the data from disk. We can actually see this logic without running the compute with the `dak.necessary_columns` function:

In [17]:
dak.necessary_columns(result)

{'from-parquet-22363367e196e0568b4bb0ee3fe22a56': frozenset({'scoring.basket',
            'scoring.distance'})}

We see the name of the input layer, and the names of the columns that are going to be read by that input layer.

This will also work with JSON. Awkward-Array's `from_json` has a feature that allows users to pass in a JSONSchema that instructs the reader which parts of the JSON dataset should be read. The reader still has to process all of the bytes in the text based file format but with a schema declared, the reader can intelligently skip over different keys in the JSON, saving memory and and time during array building.

Here's the same computation but starting with a JSON dataset:

In [18]:
dataset = dak.from_json(os.path.join("data", "json"))
free_throws = dak.str.match_substring(dataset.scoring.basket, "freethrow")
distances = dataset.scoring.distance[free_throws == False]
result = dak.mean(distances, axis=1)

In [19]:
dak.necessary_columns(result)

{'from-json-files-9fac230b2c1a3ae999b43a369b97b305': frozenset({'scoring.basket',
            'scoring.distance'})}

We see the exact same necessary columns.

A final little detail. The way that we generate the JSON schema which is then passed to the reading node is with `dak.layout_to_jsonschema`. Once the column/buffer optimization has determined which are the fields will be necessary, we can select those fields from the awkward array form that we start with after the `dak.from_json` call. We then generate an awkward array layout from the sub-form generated by selecting a subset of the columns. Finally, we create a JSONSchema from that layout:

In our small example case here, we know the columns are `scoring.basket` and `scoring.distance`. We can show this step manually here (starting with the first array collection created with the `dak.from_json call):

In [20]:
# create the subform based on the columns we need:
subform = dataset.form.select_columns(["scoring.basket", "scoring.distance"])
# create an awkward array layout:
sublayout = subform.length_zero_array(highlevel=False)
# and convert that to JSONSchema:
necessary_schema = dak.layout_to_jsonschema(sublayout)

In [21]:
necessary_schema

{'title': 'untitled',
 'description': 'Auto generated by dask-awkward',
 'type': 'object',
 'properties': {'scoring': {'type': 'array',
   'items': {'type': 'object',
    'properties': {'basket': {'type': 'string'},
     'distance': {'type': 'number'}}}}}}

This feature can be turned off when running dask-awkward graphs with the config parameter "awkward.optimization.enabled". By default this setting is `True`. We can run the same compute with the feature turned off via:

In [22]:
import dask

with dask.config.set({"awkward.optimization.enabled": False}):
    result.compute()

This could be useful for debugging. If the compute fails with the optimization enabled, but succeeds with the optimization disabled, then there is likely a bug in dask-awkward or awkward-array that should be raised!

## III. More Advanced Features

All of the high level file format readers in `dask-awkward` are based on a lower level API: the `from_map` function. This function provides an interface that allows any user defined function to be used as a source of awkward arrays at the nodes in a Dask graph.

A very simple usage of the `from_map` API would be to re-create `from_parquet`:

```python
dak.from_map(
    ak.from_parquet,
    ["/path/to/some/file1.parquet", "/path/to/some/file2.parquet"],
    label="my-from-parquet",
)
```

This will create a `dask-awkward` collection that calls `ak.from_parquet` on those two files, which as stated above, is a simple recreation of `dak.from_parquet` (obviously less flexible/powerful than `from_parquet`! but one should get the idea)

The power of `from_map` materializes when one would like to take advantage of column optimization or gracefully fail, returning an empty array instead of a program crashing, at some nodes where read issues surface. We can begin to demonstrate these features by defining a function class to be passed in as the first argument to `from_map`.

Our example will be a special Parquet reader that rejects any file that contains a "0" in the filename. For some reason we've found that data to be corrupt, but we want to be able still process the whole directory and not manually skip those files

We'll write out the class implementation and then explain each of the methods:

In [23]:
from typing import Any
from dask_awkward.lib.io.columnar import ColumnProjectionMixin

class Ignore0ParquetReader(ColumnProjectionMixin):
    def __init__(
        self,
        form: Form,
        report: bool = False,
        allowed_exceptions: tuple[type[BaseException], ...] = (OSError,),
        columns: list[str] | None = None,
        behavior: dict | None = None,
        **kwargs: Any
    ):
        self.form = form
        self.report = report
        self.allowed_exceptions = allowed_exceptions
        self.columns = columns
        self.behavior = behavior
        self.kwargs = kwargs

    @property
    def return_report(self) -> bool:
        return self.report

    @property
    def use_optimization(self) -> bool:
        return True

    @staticmethod
    def report_success(source, columns) -> ak.Array:
        return ak.Array([{"source": source, "exception": None, "columns": columns}])

    @staticmethod
    def report_failure(source, exception) -> ak.Array:
        return ak.Array([{"source": source, "exception": repr(exception), "columns": None}])

    def mock(self) -> ak.Array:
        return ak.typetracer.typetracer_from_form(self.form, highlevel=True)

    def mock_empty(self, backend="cpu") -> ak.Array:
        return ak.to_backend(self.form.length_one_array(highlevel=False), backend=backend, highlevel=True)

    def read_from_disk(self, source: Any) -> ak.Array:
        if "0" in source:
            raise OSError("cannot read files that contain '0' in the name")
        return ak.from_parquet(source, columns=self.columns, **self.kwargs)

    def __call__(self, *args, **kwargs):
        source = args[0]
        if self.return_report:
            try:
                array = self.read_from_disk(source)
                return array, self.report_success(source, self.columns)
            except self.allowed_exceptions as err:
                array = self.mock_empty()
                return array, self.report_failure(source, err)
        else:
            return self.read_from_disk(source)        

    def project_columns(self, columns):
        return Ignore0ParquetReader(
            form=self.form.select_columns(columns),
            report=self.return_report,
            allowed_exceptions=self.allowed_exceptions,
            columns=columns,
            **self.kwargs,
        )


def my_read_parquet(path, columns=None, allowed_exceptions=(OSError,)):
    pq_files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith("parquet")]
    meta_from_pq = ak.metadata_from_parquet(pq_files)
    form = meta_from_pq["form"]
    fn = Ignore0ParquetReader(form, report=True, allowed_exceptions=allowed_exceptions)
    return dak.from_map(fn, pq_files)

Here's why we have each of the methods!

- Starting with inheriting the `ColumnProjectionMixin`, inheriting from this mixin makes the class compatible with column optimization.
- `__init__`: of course this is needed. It's going to take the starting form that the array should have, a tuple of exceptions that will be allowed to be raised at compute time that we can gracefully absorb, the columns to read, the awkward-array behavior that should be used, and additional kwargs that should be passed at each node's call of `ak.from_parquet
- `return_report`: a class property that will tell `from_map` whether or not we will also return a report array
- `use_optimization`: a class property that tells the columns optimization that we want this function class to be columns optimizable.
- `report_success`: a static method that will be used to construct an report array when the read is successful at a partition
- `report_failure`: the parter to `report_success`, if one of the allowed exceptions is raised at a partition at array creation time, this method will be called to construct an report array
- `mock`: a method that "mocks" the array that would be created, returns a dataless typetracer array
- `mock_empty`: a method that mocks the array but is not a typetracer array, it's an empty concrete awkward array. This is the method that is used at nodes that fail with an allowed exception.
- `read_from_disk`: this is the method that will be called to... read data from disk! What actually matters more is the next method:
- `__call__`: we finally get to the "function" part of this class: This method will be called at each partition. You'll notice that we call `read_from_disk` here, but we wrap it in a `try`, `except` block if we want to return the read-report that allows for graceful fails
- `project_columns`: this method is necessary for rewriting the class instructing it to read a new set of columns. This method is part of the optimization interface

Finally, we write a function that is going to use this function class and call `from_map

Let's use it to read our parquet dataset and look at both the resulting array and the post-compute report. Notice that the report itself is a lazily evaluated dask-awkward Array collection that should be computed simultaneously with the collection-of-interest.

In [24]:
dataset, report = my_read_parquet(pq_dir)

In [25]:
result, computed_report = dask.compute(dataset, report)

In [26]:
result

<Array [{type: '', scoring: []}, ..., {...}] type='151 * {type: string, sco...'>

In [27]:
computed_report.tolist()

[{'source': 'data/parquet/part0.parquet',
  'exception': 'OSError("cannot read files that contain \'0\' in the name")',
  'columns': None},
 {'source': 'data/parquet/part1.parquet',
  'exception': None,
  'columns': ['scoring.basket', 'scoring.player', 'type', 'scoring.distance']},
 {'source': 'data/parquet/part3.parquet',
  'exception': None,
  'columns': ['scoring.basket', 'scoring.player', 'type', 'scoring.distance']},
 {'source': 'data/parquet/part2.parquet',
  'exception': None,
  'columns': ['scoring.basket', 'scoring.player', 'type', 'scoring.distance']}]

We can see in the report that the file with a "0" in the name indeed failed!

You'll see that we added the columns that are read to the report as well, so if we perform a compute that will only need a subset of the columns, we can get confirmation from our report array. We get the column optimization by inheriting from the column optimization mixin!

In [28]:
dak.necessary_columns(dataset.scoring.player)

{'<__main__.Ignore0ParquetReader object at 0x12fc76a-16ac8dc3f12c737b58ea50a2484e71b5': frozenset({'scoring.player'})}

In [29]:
result, computed_report= dask.compute(dataset.scoring.player, report)

In [30]:
computed_report.tolist()

[{'source': 'data/parquet/part0.parquet',
  'exception': 'OSError("cannot read files that contain \'0\' in the name")',
  'columns': None},
 {'source': 'data/parquet/part1.parquet',
  'exception': None,
  'columns': ['scoring.player']},
 {'source': 'data/parquet/part3.parquet',
  'exception': None,
  'columns': ['scoring.player']},
 {'source': 'data/parquet/part2.parquet',
  'exception': None,
  'columns': ['scoring.player']}]